# Taller sobre clustering aglomerativo y DBSCAN
# Agente Come Frutas

## Integrantes: Ayala Ivonne, Cumbal Mateo, Garcés Boris, Morales David, Pereira Alicia



### 1. Introducción
El presente informe detalla el proceso de diseño, implementación y experimentación para el desarrollo de un agente autónomo de Inteligencia Artificial en el marco del proyecto "Agente Come-Frutas". El desafío se centra en un problema clásico de toma de decisiones secuenciales en un entorno dinámico y con riesgos, sirviendo como un caso de estudio práctico para la aplicación de técnicas avanzadas de Machine Learning.

El problema consiste en un entorno de rejilla de 5x5 en el que un agente debe aprender a navegar de manera eficiente. El objetivo principal es maximizar una puntuación recolectando "frutas" (que otorgan recompensas positivas) y evitando "venenos" (que imponen castigos negativos). La meta final es desarrollar una política de comportamiento óptima que permita al agente limpiar el tablero de todas las frutas, garantizando su supervivencia al esquivar todos los venenos presentes.

Para alcanzar este objetivo, el proyecto transitó por un riguroso proceso de experimentación, explorando múltiples paradigmas de la Inteligencia Artificial. Se inició con un enfoque en el Aprendizaje por Refuerzo (Reinforcement Learning), implementando y depurando algoritmos de vanguardia como Deep Q-Networks (DQN) y su variante mejorada, Double DQN (DDQN).

Frente a los desafíos clásicos de convergencia y estabilidad inherentes a RL, la investigación se expandió para incluir otras estrategias. Se exploraron los Algoritmos Genéticos, un enfoque basado en principios de evolución, y el Aprendizaje por Imitación, una potente técnica de aprendizaje supervisado que requirió el desarrollo de un "oráculo" experto basado en el algoritmo de búsqueda A*.

A continuación, se presenta el código documentado de la implementación final, reflejando la culminación de este profundo y multifacético proceso de desarrollo.

### 2. Desarrollo y Experimentación
El proyecto se desarrolló de forma iterativa, comenzando con un algoritmo fundamental y avanzando hacia técnicas más complejas para superar los desafíos encontrados.

#### 2.1. Punto de Partida: Q-Learning

El desarrollo del agente autónomo inició con la implementación del algoritmo clásico de Aprendizaje por Refuerzo: Q-Learning. Este enfoque permitió construir una base conceptual sólida para comprender cómo un agente puede aprender a actuar en un entorno dinámico sin conocimiento previo del mismo. A través de ensayo y error, el agente aprende una política óptima que maximiza su recompensa acumulada al recolectar frutas y evitar venenos.

**Entorno y Representación de Estados** 

El entorno fue modelado como una cuadrícula de 5×5, donde cada celda puede contener una fruta (recompensa positiva), un veneno (castigo fuerte) o estar vacía. El estado del agente se representa únicamente por su posición (x,y) en la rejilla, lo que resulta en un espacio de estados de tamaño 25 (5 filas × 5 columnas).

In [ ]:
self.q_table = np.zeros((num_estados[0], num_estados[1], num_acciones))

Cada entrada de la tabla Q[s][a] almacena el valor esperado de realizar la acción a en el estado s. El agente puede realizar cuatro acciones posibles: arriba, abajo, izquierda y derecha.


**Parámetros del Aprendizaje**

Se definieron los siguientes hiperparámetros para controlar el proceso de aprendizaje:

In [ ]:
# --- PARÁMETROS DEL APRENDIZAJE POR REFUERZO (Q-LEARNING) ---
# Estos son los "hiperparámetros" que controlan cómo aprende el agente.
RECOMPENSA_FRUTA = 100         # Puntuación alta por encontrar una fruta.
CASTIGO_VENENO = -100           # Castigo fuerte por tocar un veneno.
RECOMPENSA_MOVIMIENTO = -0.1    # Castigo por movimiento para la eficiencia.
ALPHA = 0.1                     # Tasa de aprendizaje.
GAMMA = 0.9                     # Factor de descuento.
EPSILON = 1.0                   # Tasa de exploración inicial.
EPSILON_DECAY = 0.9995          # Factor de decaimiento de epsilon.
MIN_EPSILON = 0.01              # Mínima tasa de exploración.
NUM_EPISODIOS_ENTRENAMIENTO = 20000 #Número de partidas que jugará el agente para aprender.

- ALPHA define cuánto se actualiza la tabla Q con la nueva información.

- GAMMA determina la importancia del valor futuro frente a la recompensa inmediata.

- EPSILON regula el equilibrio entre exploración y explotación, y su decaimiento permite al agente ser curioso al inicio y luego explotar lo aprendido.

**Política de Acción: Epsilon-Greedy**

El agente selecciona sus movimientos mediante una política epsilon-greedy:

In [ ]:
def elegir_accion(self, estado):
    if random.uniform(0, 1) < self.epsilon:
        return random.randint(0, self.num_acciones - 1)  # Exploración
    else:
        return np.argmax(self.q_table[estado])          # Explotación


Esta estrategia permite al agente explorar nuevas acciones al inicio del entrenamiento, pero gradualmente se convierte en un experto que toma decisiones óptimas.

**Aprendizaje con la Ecuación de Bellman**

La función de actualización aplica la ecuación de Bellman, actualizando el valor de la acción ejecutada según la recompensa recibida y el mejor valor futuro estimado:

In [ ]:
def actualizar_q_table(self, estado, accion, recompensa, nuevo_estado):
    valor_antiguo = self.q_table[estado][accion]
    valor_futuro_maximo = np.max(self.q_table[nuevo_estado])
    nuevo_q = valor_antiguo + ALPHA * (
        recompensa + GAMMA * valor_futuro_maximo - valor_antiguo
    )
    self.q_table[estado][accion] = nuevo_q

Este mecanismo es el núcleo del aprendizaje en Q-Learning: el agente se adapta en función de la retroalimentación recibida del entorno.

**Entrenamiento del Agente**

El agente se entrena a lo largo de 20,000 episodios, en los que explora y aprende sobre el entorno. Cada episodio se reinicia desde la posición inicial y termina cuando el agente recolecta todas las frutas o toca un veneno.

In [ ]:
for episodio in range(NUM_EPISODIOS_ENTRENAMIENTO):
    entorno.frutas = frutas_iniciales.copy()
    entorno.venenos = venenos_iniciales.copy()
    estado = entorno.reset_a_configuracion_inicial()
    terminado = False
    while not terminado:
        accion = agente.elegir_accion(estado)
        nuevo_estado, recompensa, terminado = entorno.step(accion, "TRAINING")
        agente.actualizar_q_table(estado, accion, recompensa, nuevo_estado)
        estado = nuevo_estado
    agente.decaimiento_epsilon()

**Visualización y Modo Juego**

Una vez entrenado, el agente puede ser probado en el modo PLAYING, donde actúa usando únicamente explotación (ϵ=0). El entorno fue diseñado con pygame, incluyendo sprites para representar frutas, venenos y paredes, lo cual facilita visualizar las decisiones del agente en tiempo real.

**Limitaciones Observadas**

Aunque este enfoque demostró ser funcional en escenarios simples, presentó problemas de convergencia y eficiencia cuando el número de frutas y venenos aumentaba. La tabla Q escala mal en entornos con más dimensiones, lo que motivó la transición hacia métodos más avanzados como Deep Q-Networks.

#### 2.2 DQN con CNN (Deep Q-Network con Red Convolucional)

**Motivación y Limitaciones del Q-Learning**

Tras observar las limitaciones del Q-Learning clásico —en particular su incapacidad para escalar a entornos complejos o generalizar eficientemente— se adoptó el algoritmo Deep Q-Network (DQN) como siguiente paso evolutivo. Este enfoque reemplaza la tabla Q discreta por una red neuronal convolucional (CNN), capaz de aproximar los valores Q mediante aprendizaje profundo a partir de representaciones visuales del entorno.

**Representación del Estado como Imagen Multicanal**

Cada estado del entorno fue transformado en un tensor tridimensional de forma (3, 5, 5), donde:

- Canal 0 indica la posición del agente.

- Canal 1 marca las frutas.

- Canal 2 representa los venenos.

Esta representación permite que la CNN procese el entorno como una "imagen", reconociendo patrones espaciales.

In [ ]:
# environment.py
state = np.zeros((3, self.size, self.size), dtype=np.float32)
state[0, agent_pos[0], agent_pos[1]] = 1.0  # Canal del agente
state[1, fruta[0], fruta[1]] = 1.0         # Canal de frutas
state[2, veneno[0], veneno[1]] = 1.0       # Canal de venenos


**Arquitectura de la Red Neuronal**

La red CNN_DQN, definida en agent.py, sigue una arquitectura sencilla pero eficaz:

- 2 capas convolucionales para extraer características espaciales.

- 2 capas densas para calcular el valor Q de cada acción

In [ ]:
# Arquitectura de la red neuronal que actúa como el "cerebro" visual del agente.
class CNN_DQN(nn.Module):
    def __init__(self, h, w, outputs): [cite: 1279]
        super(CNN_DQN, self).__init__()
        # Capas convolucionales para "ver" el tabler
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        
        # Cálculo del tamaño para la capa lineal
        def conv2d_size_out(size, kernel_size=3, stride=1, padding=1):
            return (size + 2 * padding - kernel_size) // stride + 1
        
        convw = conv2d_size_out(conv2d_size_out(w))
        convh = conv2d_size_out(conv2d_size_out(h))
        linear_input_size = convw * convh * 32
        
        # Capas lineales para "decidir" el valor de cada acción
        self.fc1 = nn.Linear(linear_input_size, 256)
        self.fc2 = nn.Linear(256, outputs)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        return self.fc2(x)

Esta red recibe como entrada el estado en forma de tensor y devuelve un vector de valores Q (uno por acción).

**Componentes del Agente DQN**

El agente implementa técnicas avanzadas del algoritmo DQN según la arquitectura de Mnih et al. (2015):

a) Red principal y red objetivo
- La red principal es la que se entrena activamente.

- La red objetivo se actualiza cada cierto número de pasos (update_target_every) para estabilizar el entrenamiento.

In [ ]:
self.target_model.load_state_dict(self.model.state_dict())

b) Memoria de Replay

El agente almacena transiciones pasadas en una cola circular (deque), desde donde extrae mini-batches para entrenamiento aleatorio. Esto rompe la correlación temporal entre muestras y mejora la estabilidad.

In [ ]:
self.memory = deque(maxlen=20000)
self.memory.append((state, action, reward, next_state, done))

c) Política Epsilon-Greedy

Durante el entrenamiento, el agente decide sus acciones con una política que balancea exploración y explotación. Con probabilidad ϵ, elige una acción aleatoria; de lo contrario, toma la acción con mayor valor Q:

In [ ]:
if explore and np.random.rand() <= self.epsilon:
    return random.randrange(self.action_size)
...
action_values = self.model(state_tensor)
return np.argmax(action_values.cpu().data.numpy())

El valor de ϵ decae progresivamente para favorecer la explotación del conocimiento adquirido:

In [ ]:
if self.epsilon > self.epsilon_min:
    self.epsilon *= self.epsilon_decay

**Proceso de Entrenamiento**

El agente entrena su red principal mediante retropropagación en lotes aleatorios (batch_size) tomados de la memoria de replay. Cada lote actualiza la red usando la pérdida entre los valores actuales y los objetivos (targets):

In [ ]:
# Cálculo del valor objetivo (target Q)
target_q_values = rewards + (gamma * next_q_values * (~dones))
loss = criterion(current_q_values, target_q_values)
loss.backward()
optimizer.step()

El uso de torch.no_grad() para calcular los targets evita que se acumulen gradientes en la red objetivo.

**Interfaz Visual e Interacción**

El entorno visual fue implementado en main.py usando Pygame. Incluye dos modos:

- SETUP: permite al usuario colocar frutas y venenos manualmente en el tablero.

- RUN: el agente entrenado toma el control y ejecuta su política aprendida, visualizando su comportamiento en tiempo real.

Esto permitió una validación cualitativa del rendimiento del agente en diferentes configuraciones.

**Resultados y Observaciones**

- El uso de CNN permitió al agente generalizar mejor en diferentes escenarios.

- El sistema de reward shaping (recompensas por acercarse a frutas) mejoró la eficiencia del entrenamiento.

- La representación como imagen multicanal ayudó a la red a reconocer patrones espaciales útiles para la navegación.

#### 2.3 DDQN

**Problemas con DQN clásico**

Aunque Deep Q-Networks (DQN) mejoró la generalización del agente frente al Q-Learning tradicional, aún sufría de un problema conocido como overestimation bias: la tendencia a sobrevalorar las recompensas futuras. Esto ocurre porque DQN utiliza la misma red para seleccionar y evaluar las acciones futuras, lo que puede introducir errores acumulativos.

Para mitigar este problema, se implementó el algoritmo Double DQN (DDQN), una mejora propuesta por Van Hasselt et al. (2016), que desacopla el proceso de selección de acción del de evaluación de valor. Esto reduce el sesgo y mejora la estabilidad del entrenamiento.

**Diferencia Clave: Selección vs Evaluación**

En DQN clásico:

In [ ]:
target = reward + gamma * max(Q_target(next_state))

En Double DQN:

In [ ]:
# Selección con red principal
next_actions = model(next_state).argmax(dim=1)

# Evaluación con red objetivo
next_q = target_model(next_state).gather(1, next_actions.unsqueeze(1))
target = reward + gamma * next_q

Este cambio sutil mejora notablemente la precisión de la estimación de los valores Q futuros.

**Implementación en el Proyecto**

La clase Agent en agent.py fue modificada para aplicar el criterio Double DQN en el método replay(). La arquitectura de la red se mantuvo similar al agente DQN, pero se ajustó la forma en que se calcula el valor objetivo durante el entrenamiento.

Fragmento clave del método replay:

In [ ]:
# 1. Selección con la red principal
next_actions = self.model(next_states).detach().argmax(dim=1).unsqueeze(1)

# 2. Evaluación con la red objetivo
next_q_values = self.target_model(next_states).gather(1, next_actions)

# 3. Cálculo de target
targets = rewards + gamma * next_q_values * (1 - dones)


**Otras Mejoras y Técnicas Usadas**

Además del cambio en el cálculo de los valores objetivos, el agente DDQN mantiene:

- Una política epsilon-greedy con decaimiento gradual.

- Replay Buffer circular para almacenar transiciones de entrenamiento.

- Actualización periódica de la red objetivo para estabilizar el aprendizaje.

- Entrenamiento mini-batch con retropropagación en train.py.

**Interfaz Interactiva**

La demostración del agente DDQN se integró a través de main.py, mientras que el entrenamiento puede ser gestionado desde interfaztrain.py. La representación visual de frutas, venenos y el agente permite observar cómo la política aprendida se adapta al entorno.

**Resultados Obtenidos**

Durante el entrenamiento de DDQN, se observó una mejora significativa en:

- Estabilidad del aprendizaje: menos oscilaciones durante el entrenamiento.

- Precisión en la toma de decisiones: el agente evitó más venenos sin necesidad de aumentar la exploración.

- Generalización: el agente entrenado pudo actuar correctamente en escenarios no vistos durante el entrenamiento.

#### 2.4. Paradigma Alternativo: Algoritmos Genéticos

**Motivación**

A diferencia de los algoritmos anteriores que dependen de gradientes y retropropagación (como Q-Learning o DQN), el enfoque evolutivo busca optimizar directamente las políticas de los agentes mediante la simulación de un proceso de selección natural. Esta técnica resulta especialmente útil cuando se trabaja con entornos ruidosos o no diferenciables, donde las funciones de pérdida tradicionales pueden fallar o llevar a soluciones subóptimas.

**Fundamentos del Algoritmo Genético**

El algoritmo genético implementado se basa en los principios fundamentales de la evolución biológica:

1. Inicialización: Se genera una población aleatoria de agentes, cada uno con una red neuronal que actúa como su "ADN".

2. Evaluación (Fitness): Cada agente se enfrenta a un entorno dinámico y su desempeño se mide como la suma de recompensas obtenidas.

3. Selección: Se eligen los agentes con mayor fitness para reproducirse.

4. Cruzamiento (Crossover): Los genes de dos padres se combinan para formar un nuevo agente hijo.

5. Mutación: Se introduce variación aleatoria en los genes del hijo.

6. Elitismo y Reemplazo: Se conservan los mejores agentes y se forma una nueva generación con los hijos generados.

Este ciclo se repite durante múltiples generaciones hasta que el agente evoluciona una política óptima.

**Implementación del Agente Genético**

La arquitectura general del entrenamiento se encuentra en train_ga.py, y el comportamiento del agente está definido en agent_ga.py.

**a) Estructura de la Población**

Cada agente está representado por una red neuronal densa que recibe el estado del entorno y genera una acción discreta. Los pesos de esta red constituyen su genotipo. Inicialmente, los pesos son asignados aleatoriamente:

In [ ]:
def create_initial_population():
    return [Agent() for _ in range(POPULATION_SIZE)]


**b) Evaluación de Fitness**

Cada agente es evaluado en un escenario aleatorio donde debe recolectar frutas y evitar venenos. Su desempeño se mide por la suma total de recompensas:

In [ ]:
for _ in range(50):
    action = agent.choose_action(state)
    state, reward, done = env.step(action)
    total_reward += reward

Este total es almacenado como su fitness, y determina su posibilidad de ser seleccionado como padre.

**c) Selección de Padres**

Se selecciona el 20% de los mejores agentes de la población como candidatos para reproducción:

In [ ]:
population.sort(key=lambda x: x.fitness, reverse=True)
parents = population[:int(POPULATION_SIZE * 0.2)]

**d) Cruzamiento Uniforme**

La recombinación de genes se realiza de manera uniforme: cada peso tiene una probabilidad del 50% de heredarse de cada padre:

In [ ]:
for key in child_genes.keys():
    if random.random() < 0.5:
        child_genes[key] = p1_genes[key].clone()
    else:
        child_genes[key] = p2_genes[key].clone()

**e) Mutación Gaussiana**

Cada parámetro tiene una probabilidad de ser alterado añadiendo ruido gaussiano con desviación estándar 0.1, lo que permite explorar nuevas soluciones:

In [ ]:
noise = torch.randn_like(child_genes[key]) * 0.1
child_genes[key] += noise

#### 2.5 Solución Definitiva: Aprendizaje por Imitación y Currículo

Tras los desafíos encontrados, la estrategia final y más exitosa fue el Aprendizaje por Imitación. Este enfoque cambia el paradigma: en lugar de que el agente descubra una estrategia por sí mismo, se le enseña directamente imitando a un experto perfecto. Para ello, se implementó el algoritmo de búsqueda A* como un "oráculo" capaz de calcular siempre la ruta óptima en cualquier tablero.

El primer paso fue generar un "libro de texto" para el agente, creando miles de ejemplos de jugadas perfectas. Se crearon datasets separados para escenarios de 1, 2, 3 y 4 frutas.

In [ ]:
# Función clave para generar los datos de entrenamiento para el currículo.
def generate_expert_data_for_n_fruits(num_fruits, num_samples, output_file):
    """Genera un dataset experto para un número específico de frutas."""
    env = GridEnvironment()
    expert_data = []
    print(f"Generando {num_samples} muestras para {num_fruits} fruta(s)...")
    
    while len(expert_data) < num_samples:
        # Generar un escenario aleatorio con `num_fruits` frutas.
        # ... (código de generación de escenario) ...
        env.reset(agent_pos=agent_p, fruit_pos=fruit_p, poison_pos=poison_p)
        
        # Simular el episodio usando el experto A* paso a paso.
        for _ in range(50):
            if not env.fruit_pos: break
            
            # El experto A* recalcula la mejor ruta desde la posición actual.
            path = a_star_search(GRID_SIZE, env.agent_pos, goal_fruit, env.poison_pos)

            if path and len(path) > 0:
                # Se toma solo el primer paso de la ruta óptima.
                action = get_action(env.agent_pos, path[0])
                state = env.get_state()
                # Se guarda el par (estado_actual, accion_correcta).
                expert_data.append((state, action))
                env.step(action)
            else:
                break
                
    with open(output_file, "wb") as f:
        pickle.dump(expert_data, f)
    print(f"Dataset '{output_file}' creado.")

La innovación crucial fue entrenar al agente con un Aprendizaje por Currículo. En lugar de mostrarle todos los datos a la vez, el agente fue entrenado por etapas, comenzando con la tarea más fácil (1 fruta) y aumentando gradualmente la dificultad. Esto le permitió dominar primero los conceptos básicos de "ganar" antes de enfrentarse a escenarios más complejos.

In [ ]:
# El script de entrenamiento que implementa la estrategia de Curriculum Learning.
if __name__ == "__main__":
    model = AgentNetwork()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()

    # El Currículo: una lista de "lecciones" de dificultad creciente.
    CURRICULUM = [
        ("expert_data_1_fruit.pkl", 25),   # Lección 1: Nivel básico
        ("expert_data_2_fruits.pkl", 30),  # Lección 2: Nivel intermedio
        ("expert_data_3_fruits.pkl", 40),  # Lección 3: Nivel avanzado
        ("expert_data_4_fruits.pkl", 50)   # Lección 4: Nivel experto
    ]

    # Bucle principal que itera a través de las lecciones del currículo.
    for i, (dataset_file, num_epochs) in enumerate(CURRICULUM):
        print(f"\n--- Iniciando Lección {i+1}/{len(CURRICULUM)}: {dataset_file} ---")
        
        # Cargar el dataset de la lección actual.
        with open(dataset_file, "rb") as f:
            data = pickle.load(f)
        
        # Preparar los datos para el entrenamiento.
        # ... (código de creación del DataLoader) ...

        # Entrenar el modelo (sin reiniciarlo) por el número de épocas de la lección.
        for epoch in range(num_epochs):
            # ... (bucle de entrenamiento estándar de aprendizaje supervisado) ...
            print(f"  Época {epoch+1}/{num_epochs}, Pérdida: {avg_loss:.4f}")

    # Guardar el modelo final, que ha aprendido de todo el currículo.
    torch.save(model.state_dict(), "imitacion_model.pth")

### Jugador humano

In [ ]:
"""
Modo de juego humano con controles aleatorios - Come Frutas.

Este módulo implementa una versión jugable del entorno donde un humano puede
controlar el agente directamente. La característica única es que los controles
de movimiento se asignan aleatoriamente cada vez que se inicia una partida,
añadiendo un elemento de desafío y adaptabilidad.

Características principales:
- Modo Setup: Configuración manual del escenario
- Modo Humano: Control directo del agente por el jugador
- Controles aleatorios: Mapeo aleatorio de teclas a movimientos
- Interfaz intuitiva: Gráficos y feedback visual
- Desafío adaptativo: Cada partida requiere aprender nuevos controles

Propósito educativo:
- Comparar rendimiento humano vs. IA
- Experimentar la dificultad de adaptación a controles cambiantes
- Entender la importancia de la consistencia en interfaces
- Apreciar la flexibilidad del aprendizaje humano

Autor: [Tu nombre]
Fecha: Agosto 2025
"""

import pygame
import os
import random
import string

# CONFIGURACIÓN DEL ENTORNO VISUAL
"""
Parámetros visuales y dimensiones de la interfaz de juego.
Utiliza celdas más grandes (120px) para mejor visibilidad durante el juego manual.
"""
GRID_WIDTH = 5              # Ancho de la cuadrícula en celdas
GRID_HEIGHT = 5             # Alto de la cuadrícula en celdas
CELL_SIZE = 120             # Tamaño de cada celda en píxeles (mayor para juego manual)
SCREEN_WIDTH = GRID_WIDTH * CELL_SIZE    # Ancho total de la ventana (600px)
SCREEN_HEIGHT = GRID_HEIGHT * CELL_SIZE  # Alto del área de juego (600px)

# PALETA DE COLORES CONSISTENTE
"""
Esquema de colores oscuro profesional, consistente con otros módulos del proyecto.
"""
COLOR_FONDO = (25, 25, 25)      # Gris muy oscuro para el fondo
COLOR_LINEAS = (40, 40, 40)     # Gris oscuro para líneas de cuadrícula
COLOR_CURSOR = (255, 255, 0)    # Amarillo brillante para cursor de selección
COLOR_TEXTO = (230, 230, 230)   # Gris claro para texto legible

# SISTEMA DE CONTROLES ALEATORIOS
"""
Genera un conjunto de teclas válidas para asignación aleatoria de controles.
Se evitan teclas especiales para prevenir conflictos con funciones del sistema.
"""
TECLAS_VALIDAS = [getattr(pygame, f"K_{c}") for c in string.ascii_lowercase + string.digits]

class EntornoHumano:
    """
    Entorno de juego optimizado para control humano directo.
    
    Esta clase maneja la lógica del juego cuando un humano controla el agente,
    incluyendo movimiento, colisiones, recolección de objetos y condiciones
    de victoria/derrota. Se enfoca en proporcionar feedback inmediato y
    una experiencia de juego fluida.
    
    Diferencias con entornos de IA:
    - Feedback inmediato con mensajes en consola
    - Lógica de juego simplificada (sin recompensas numéricas)
    - Terminación inmediata en victoria/derrota
    - Controles responsivos para jugabilidad humana
    
    Attributes:
        agente_pos (tuple): Posición actual del agente (x, y)
        frutas (set): Conjunto de posiciones con frutas
        venenos (set): Conjunto de posiciones con venenos
        paredes (set): Conjunto de posiciones con paredes/obstáculos
    """
    def __init__(self):
        """
        Inicializa el entorno con configuración vacía.
        
        El agente comienza en la esquina superior izquierda (0,0) y todos
        los conjuntos de elementos están vacíos, permitiendo configuración manual.
        """
        self.agente_pos = (0, 0)    # Posición inicial del agente
        self.frutas = set()         # Conjunto de posiciones de frutas
        self.venenos = set()        # Conjunto de posiciones de venenos
        self.paredes = set()        # Conjunto de posiciones de paredes

    def reset(self):
        """
        Resetea la posición del agente al inicio del juego.
        
        Coloca al agente en la posición inicial (0,0) sin modificar
        la configuración del escenario. Utilizado al comenzar una nueva partida.
        """
        self.agente_pos = (0, 0)

    def limpiar(self):
        """
        Elimina todos los elementos del entorno.
        
        Limpia frutas, venenos y paredes del escenario, dejando una
        cuadrícula vacía para configuración desde cero.
        """
        self.frutas.clear()
        self.venenos.clear()
        self.paredes.clear()

    def step(self, accion):
        """
        Ejecuta una acción del jugador humano en el entorno.
        
        Procesa el movimiento del agente, verifica colisiones y maneja
        las interacciones con elementos del entorno. Proporciona feedback
        inmediato al jugador mediante mensajes en consola.
        
        Args:
            accion (int): Dirección de movimiento:
                         0 = Arriba (decrementar y)
                         1 = Abajo (incrementar y)
                         2 = Izquierda (decrementar x)
                         3 = Derecha (incrementar x)
        
        Returns:
            bool: True si el juego terminó (victoria o derrota), False si continúa
        """
        # Calcular nueva posición basada en la acción
        x, y = self.agente_pos
        if accion == 0:     # Arriba
            y -= 1
        elif accion == 1:   # Abajo
            y += 1
        elif accion == 2:   # Izquierda
            x -= 1
        elif accion == 3:   # Derecha
            x += 1

        # Verificar colisiones: límites del tablero o paredes
        if x < 0 or x >= GRID_WIDTH or y < 0 or y >= GRID_HEIGHT or (x, y) in self.paredes:
            # Movimiento inválido: no actualizar posición
            return False

        # Movimiento válido: actualizar posición del agente
        self.agente_pos = (x, y)
        
        # Verificar interacciones con elementos del entorno
        if self.agente_pos in self.frutas:
            # Fruta recogida: eliminar del conjunto
            self.frutas.remove(self.agente_pos)
            
            # Verificar condición de victoria
            if not self.frutas:
                print("\n✨ ¡Ganaste! Recolectaste todas las frutas.\n")
                return True  # Juego terminado con éxito
                
        elif self.agente_pos in self.venenos:
            # Veneno tocado: derrota inmediata
            print("\n☠️ ¡Oh no! Tocaste un veneno.\n")
            return True  # Juego terminado con fallo
            
        # Continuar juego
        return False

    def dibujar(self, pantalla, modo, cursor_pos, img_fruta, img_veneno, img_pared, img_agente, _):
        """
        Renderiza el estado completo del entorno con interfaz interactiva.
        
        Dibuja todos los elementos visuales del juego incluyendo grid, objetos
        del entorno y cursor de selección. Proporciona feedback visual para
        la interacción del jugador en diferentes modos (colocación/juego).
        
        Args:
            pantalla (pygame.Surface): Superficie donde renderizar
            modo (str): Modo actual de la interfaz ('frutas', 'venenos', 'paredes', 'jugar')
            cursor_pos (tuple): Posición (x,y) del cursor en coordenadas de grid
            img_fruta (pygame.Surface): Sprite de las frutas
            img_veneno (pygame.Surface): Sprite de los venenos
            img_pared (pygame.Surface): Sprite de las paredes
            img_agente (pygame.Surface): Sprite del agente
            _ : Parámetro no utilizado (compatibilidad de interfaz)
        
        Note:
            Renderiza en orden específico: fondo, grid, objetos, agente, cursor.
            El cursor cambia de color según el modo de colocación activo.
        """
        # Limpiar pantalla con color de fondo
        pantalla.fill(COLOR_FONDO)

        # Dibujar líneas del grid para guía visual
        for x in range(0, SCREEN_WIDTH, CELL_SIZE):
            pygame.draw.line(pantalla, COLOR_LINEAS, (x, 0), (x, SCREEN_HEIGHT))
        for y in range(0, SCREEN_HEIGHT, CELL_SIZE):
            pygame.draw.line(pantalla, COLOR_LINEAS, (0, y), (SCREEN_WIDTH, y))

        # Dibujar elementos del entorno
        for fruta in self.frutas:
            pantalla.blit(img_fruta, (fruta[0]*CELL_SIZE, fruta[1]*CELL_SIZE))
        for veneno in self.venenos:
            pantalla.blit(img_veneno, (veneno[0]*CELL_SIZE, veneno[1]*CELL_SIZE))
        for pared in self.paredes:
            pantalla.blit(img_pared, (pared[0]*CELL_SIZE, pared[1]*CELL_SIZE))

        # Dibujar agente (jugador) - siempre visible en primer plano
        pantalla.blit(img_agente, (self.agente_pos[0]*CELL_SIZE, self.agente_pos[1]*CELL_SIZE))

        # Dibujar cursor de selección en modo configuración
        if modo == "SETUP":
            cursor_rect = pygame.Rect(cursor_pos[0]*CELL_SIZE, cursor_pos[1]*CELL_SIZE, CELL_SIZE, CELL_SIZE)
            pygame.draw.rect(pantalla, COLOR_CURSOR, cursor_rect, 3)

        # Renderizar información de interfaz
        font = pygame.font.Font(None, 30)
        pantalla.blit(font.render(f"Modo: {modo}", True, COLOR_TEXTO), (10, SCREEN_HEIGHT + 5))
        pantalla.blit(font.render("F: Fruta, V: Veneno, W: Pared, C: Limpiar, H: Jugar", True, COLOR_TEXTO), (10, SCREEN_HEIGHT + 30))
        pantalla.blit(font.render("Descubre los controles ocultos usando letras/números", True, COLOR_TEXTO), (10, SCREEN_HEIGHT + 55))

def cargar_imagen(nombre, fallback_color):
    """
    Carga una imagen desde archivo con sistema de respaldo.
    
    Intenta cargar una imagen sprite desde el directorio actual.
    Si la carga falla, crea una superficie de color sólido como respaldo.
    Escala automáticamente al tamaño de celda definido.
    
    Args:
        nombre (str): Nombre del archivo de imagen a cargar
        fallback_color (tuple): Color RGB (r,g,b) para superficie de respaldo
    
    Returns:
        pygame.Surface: Superficie cargada y escalada, o superficie de color
                       si la carga falló
    
    Note:
        Todas las imágenes se escalan a CELL_SIZE x CELL_SIZE píxeles.
        Utiliza convert_alpha() para optimizar el renderizado con transparencia.
    """
    try:
        # Construir ruta completa al archivo de imagen
        ruta = os.path.join(os.path.dirname(__file__), nombre)
        # Cargar imagen con soporte de transparencia
        img = pygame.image.load(ruta).convert_alpha()
        # Escalar a tamaño de celda estándar
        return pygame.transform.scale(img, (CELL_SIZE, CELL_SIZE))
    except:
        # Crear superficie de respaldo con color sólido si falla la carga
        s = pygame.Surface((CELL_SIZE, CELL_SIZE))
        s.fill(fallback_color)
        return s

def generar_controles_aleatorios():
    """
    Genera un mapeo aleatorio de teclas para controles de movimiento.
    
    Crea una asignación aleatoria entre teclas del teclado y direcciones
    de movimiento para añadir un elemento de desafío y descubrimiento
    al juego. Los jugadores deben encontrar qué teclas controlan cada dirección.
    
    Returns:
        dict: Mapeo de códigos de tecla pygame a acciones de movimiento:
              {tecla_pygame: accion_int}
              donde accion_int es 0=Arriba, 1=Abajo, 2=Izquierda, 3=Derecha
    
    Note:
        Utiliza teclas alfanuméricas (A-Z, 0-9) para máxima compatibilidad.
        Garantiza que cada dirección tenga exactamente una tecla asignada.
    """
    # Seleccionar 4 teclas aleatorias del conjunto disponible
    teclas = random.sample(TECLAS_VALIDAS, 4)
    # Crear lista de acciones de movimiento
    acciones = [0, 1, 2, 3]  # Arriba, abajo, izquierda, derecha
    # Mezclar aleatoriamente las acciones
    random.shuffle(acciones)
    # Crear diccionario de mapeo tecla->acción
    return dict(zip(teclas, acciones))

def main():
    """
    Función principal del juego en modo humano.
    
    Inicializa Pygame, configura la ventana de juego y ejecuta el bucle
    principal que maneja dos modos: configuración del entorno y juego
    con controles aleatorios. Proporciona una experiencia interactiva
    donde el jugador puede diseñar niveles y luego jugarlos.
    
    Flujo del juego:
        1. Modo SETUP: Colocar frutas, venenos y paredes con el mouse
        2. Modo JUGAR: Controlar agente con teclas aleatorias descubiertas
        3. Victoria: Recolectar todas las frutas
        4. Derrota: Tocar veneno
    
    Controles SETUP:
        - Mouse: Mover cursor
        - F: Colocar fruta
        - V: Colocar veneno  
        - W: Colocar pared
        - C: Limpiar todo
        - H: Iniciar juego
    
    Controles JUGAR:
        - Teclas aleatorias para movimiento (descubrir experimentando)
        - ESC: Volver a configuración
    """
    # Inicializar Pygame y configurar ventana
    pygame.init()
    pantalla = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT + 100))
    pygame.display.set_caption("Modo Humano Aleatorio - Come Frutas")

    # Inicializar entorno y variables de estado
    entorno = EntornoHumano()
    cursor_pos = [0, 0]
    modo = "SETUP"  # Modo inicial: configuración del entorno
    mapeo_controles = {}  # Mapeo de teclas aleatorias (generado al jugar)

    # Cargar sprites con colores de respaldo
    img_fruta = cargar_imagen("fruta.png", (40, 200, 40))
    img_veneno = cargar_imagen("veneno.png", (255, 50, 50))
    img_pared = cargar_imagen("pared.jpg", (80, 80, 80))
    img_agente = cargar_imagen("agente.png", (60, 100, 255))

    # Variables de control del juego
    reloj = pygame.time.Clock()
    corriendo = True

    # Bucle principal del juego
    while corriendo:
        # Procesar eventos de entrada
        for evento in pygame.event.get():
            if evento.type == pygame.QUIT:
                corriendo = False
            elif evento.type == pygame.KEYDOWN:
                if evento.key == pygame.K_s:
                    modo = "SETUP"
                elif evento.key == pygame.K_h:
                    modo = "HUMANO"
                    entorno.reset()
                    mapeo_controles = generar_controles_aleatorios()

                if modo == "SETUP":
                    if evento.key == pygame.K_UP: cursor_pos[1] = max(0, cursor_pos[1]-1)
                    elif evento.key == pygame.K_DOWN: cursor_pos[1] = min(GRID_HEIGHT-1, cursor_pos[1]+1)
                    elif evento.key == pygame.K_LEFT: cursor_pos[0] = max(0, cursor_pos[0]-1)
                    elif evento.key == pygame.K_RIGHT: cursor_pos[0] = min(GRID_WIDTH-1, cursor_pos[0]+1)
                    # Colocación de elementos con teclas específicas
                    pos = tuple(cursor_pos)
                    if evento.key == pygame.K_f: 
                        # F: Colocar/quitar fruta (toggle)
                        entorno.frutas.symmetric_difference_update({pos})
                        entorno.venenos.discard(pos)
                        entorno.paredes.discard(pos)
                    elif evento.key == pygame.K_v: 
                        # V: Colocar/quitar veneno (toggle)
                        entorno.venenos.symmetric_difference_update({pos})
                        entorno.frutas.discard(pos)
                        entorno.paredes.discard(pos)
                    elif evento.key == pygame.K_w: 
                        # W: Colocar/quitar pared (toggle)
                        entorno.paredes.symmetric_difference_update({pos})
                        entorno.frutas.discard(pos)
                        entorno.venenos.discard(pos)
                    elif evento.key == pygame.K_c: 
                        # C: Limpiar todo el entorno
                        entorno.limpiar()

                # Controles específicos del modo HUMANO
                elif modo == "HUMANO":
                    if evento.key in mapeo_controles:
                        # Ejecutar acción de movimiento con tecla aleatoria
                        accion = mapeo_controles[evento.key]
                        terminado = entorno.step(accion)
                        if terminado:
                            # Volver a configuración al terminar el juego
                            modo = "SETUP"

        # Renderizar estado actual del juego
        entorno.dibujar(pantalla, modo, cursor_pos, img_fruta, img_veneno, img_pared, img_agente, mapeo_controles)
        pygame.display.flip()
        reloj.tick(30)

    # Limpiar recursos al salir
    pygame.quit()

if __name__ == '__main__':
    main()

### 3. Resultados y Conclusión
El viaje a través de estos diferentes paradigmas de IA fue revelador. Mientras que los enfoques de Aprendizaje por Refuerzo y Algoritmos Genéticos mostraron potencial, también exhibieron dificultades para converger de manera consistente en un entorno tan "frágil", donde un solo error puede llevar al fracaso.

La estrategia de Aprendizaje por Imitación combinada con el Aprendizaje por Currículo demostró ser la más efectiva y robusta. Al aprender de un experto perfecto y hacerlo de manera gradual, el agente final fue capaz de generalizar su conocimiento y resolver de manera confiable tanto escenarios simples como complejos. Para la demostración visual, se desarrolló una interfaz en Pygame que permite a los usuarios configurar sus propios niveles y observar al agente entrenado resolverlos en tiempo real.

Este proyecto subraya una lección fundamental en el desarrollo de IA: a menudo, la estrategia de entrenamiento y la calidad de los datos son tan o más importantes que la elección del algoritmo en sí. El resultado es un agente competente y una profunda comprensión práctica de los desafíos y soluciones en el campo del Machine Learning.